In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale, PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor

from radio_snr import *

In [3]:
df = pd.read_csv('wspr_sample.csv', index_col=0)
df.head()

,spot,timestamp,reporter,reporter_grid,snr,frequency,tx_call,tx_grid,power,drift,distance,azimuth,band,version,code
4541493,1093722523,1520245320,DC5AL-R,JO31lk,9,7.040113,G0NJS,IO91vs,37,0,496,91,7,NaN,0
4692526,1093866340,1520259000,KA3JIJ,EM84cj,-27,10.140175,WB0KSL,EM28nu,37,0,1105,113,10,NaN,0
17907988,1107180087,1521372240,PI9ESA,JO22ff,-5,10.140179,IQ6KX,JN63so,20,0,1172,328,10,NaN,0
10270295,1099476317,1520727720,PA0EHG,JO22hb,-17,3.594176,PA7MDJ,JO21is,23,0,33,350,3,NaN,0
29854808,1119235300,1522494360,AG5OV,EL09,-10,7.040137,WA4KFZ,FM18gv,37,0,2236,249,7,1.9.0-rc3,0


In [4]:
df = preprocess_data(df)
df.head()

,spot,reporter,snr,frequency,tx_call,power,drift,distance,azimuth,band,version,code,rx_lat,rx_long,tx_lat,tx_long,day,hour
4541493,1093722523,DC5AL-R,9,7.040113,G0NJS,37,0,496,91,7,NaN,0,51.437500,6.958333,51.770833,-0.208333,17595,10
4692526,1093866340,KA3JIJ,-27,10.140175,WB0KSL,37,0,1105,113,10,NaN,0,34.395833,-83.791667,38.854167,-94.875000,17595,14
17907988,1107180087,PI9ESA,-5,10.140179,IQ6KX,20,0,1172,328,10,NaN,0,52.229167,4.458333,43.604167,13.541667,17608,11
10270295,1099476317,PA0EHG,-17,3.594176,PA7MDJ,23,0,33,350,3,NaN,0,52.062500,4.625000,51.770833,4.708333,17601,0
29854808,1119235300,AG5OV,-10,7.040137,WA4KFZ,37,0,2236,249,7,1.9.0-rc3,0,29.000000,-100.000000,38.895833,-77.458333,17621,11


In [5]:
targets = df['snr']
features = df[['power', 'drift', 'distance', 'azimuth', 'band', 'rx_lat', 'rx_long', 'tx_lat', 'tx_long', 'day', 'hour']]
features.head()

,power,drift,distance,azimuth,band,rx_lat,rx_long,tx_lat,tx_long,day,hour
4541493,37,0,496,91,7,51.437500,6.958333,51.770833,-0.208333,17595,10
4692526,37,0,1105,113,10,34.395833,-83.791667,38.854167,-94.875000,17595,14
17907988,20,0,1172,328,10,52.229167,4.458333,43.604167,13.541667,17608,11
10270295,23,0,33,350,3,52.062500,4.625000,51.770833,4.708333,17601,0
29854808,37,0,2236,249,7,29.000000,-100.000000,38.895833,-77.458333,17621,11


In [10]:
features.loc[:,:] = scale(features)
features.describe()

/home/tom/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/tom/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,power,drift,distance,azimuth,band,rx_lat,rx_long,tx_lat,tx_long,day,hour
count,1.524100e+04,1.524100e+04,1.524100e+04,1.524100e+04,1.524100e+04,1.524100e+04,1.524100e+04,1.524100e+04,1.524100e+04,1.524100e+04,1.524100e+04
mean,-1.396793e-16,-6.144980e-16,8.595651e-18,-3.575208e-17,-2.369413e-16,-5.428372e-17,2.607833e-17,-1.003069e-17,6.266084e-17,1.607824e-16,-8.701640e-17
std,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00
min,-4.136937e+00,-5.526641e+00,-7.916351e-01,-1.561126e+00,-7.668448e-01,-7.236420e+00,-2.416686e+00,-9.019212e+00,-2.430953e+00,-1.613624e+00,-1.853105e+00
25%,-9.619939e-01,1.871674e-01,-4.988743e-01,-9.353673e-01,-2.357083e-01,-2.984445e-01,-1.009491e+00,-3.160563e-01,-1.027582e+00,-8.365406e-01,-7.949063e-01
50%,4.293119e-03,1.871674e-01,-3.035568e-01,-6.474661e-02,-5.866280e-02,2.926363e-01,5.091945e-01,2.213749e-01,4.197150e-01,-5.945738e-02,1.121212e-01
75%,9.705801e-01,1.871674e-01,4.482426e-02,9.872533e-01,2.069054e-01,5.133065e-01,6.146588e-01,5.356107e-01,5.863322e-01,8.286377e-01,8.679775e-01
max,3.731400e+00,5.900976e+00,7.265107e+00,1.703702e+00,4.136998e+01,1.574625e+00,3.584985e+00,2.539965e+00,3.593015e+00,1.716733e+00,1.623834e+00


In [16]:
mse = []
r2 = []
for n in range(1, 25):
    print(n)
    knn = KNeighborsRegressor(n_neighbors=n)
    tmp_mse = -1*cross_val_score(knn, features, targets, scoring='neg_mean_squared_error')
    mse.append(tmp_mse)
    knn.fit(features, targets)
    tmp_r2 = knn.score(features, targets)
    r2.append(tmp_r2)
err = pd.DataFrame()
err['rmse'] = np.sqrt(np.array(mse))
err['r^2'] = r2
err.sort_values('rmse')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series